In [1]:
import builtins
import datetime
old_print = builtins.print
 
def hook_print():
    def my_print(*args, **kwargs):
        old_print(datetime.datetime.now(), end=" ")
        old_print(*args, **kwargs)
    builtins.print = my_print
def unhook_print():
    builtins.print = old_print
hook_print()

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


# bs*in_channel*in_h*in_w
input = torch.randn(1, 1, 200, 200)
# out_channel*inchannel*kernel_h*kernel*w
kernel = torch.randn(1, 1, 3, 3)

# bias = torch.randn(3)

# 用矩阵运算实现二维卷积 考虑batch_size和channels维度
def matrix_multiplication_for_conv2d(input, kernel,bias = 0, stride=1, padding=0):
    if padding > 0:
        input = F.pad(input, (padding, padding, padding, padding, 0, 0, 0, 0))
    # 计算输出大小
    bs, in_channels, input_h, input_w = input.shape
    out_channel, in_channel,  kernel_h, kernel_w = kernel.shape

    if bias is None:
        bias = torch.zeros(out_channel)

    output_h = (math.floor((input_h - kernel_h) / stride) + 1)
    output_w = (math.floor((input_w - kernel_w) / stride) + 1)

    # 初始化输出矩阵
    output = torch.zeros(bs, out_channel, output_h, output_w)
	
    # 不考虑性能
    for ind in range(bs):
        for oc in range(out_channel):
            for ic in range(in_channel):
                for i in range(0, input_h - kernel_h + 1, stride):
                    for j in range(0, input_w - kernel_w + 1, stride):
                        region = input[ind, ic, i:i + kernel_h, j: j + kernel_w]
                        # 点乘相加
                        output[ind, oc, int(i / stride), int(j / stride)] += torch.sum(region * kernel[oc, ic])
            # output[ind, oc] += bias[oc]


    return output

print(1)

res = matrix_multiplication_for_conv2d(input, kernel, bias = bias, padding=1, stride=2)

print(2)

# 验证一下
# 调用API结果
res2 = F.conv2d(
    input,
    kernel,
    padding=1,
    # bias=bias,
    stride=2
)

print(torch.allclose(res, res2))

2023-05-08 01:21:27.812881 1
2023-05-08 01:21:27.999521 2
2023-05-08 01:21:28.000070 False


In [2]:
import math
import sys
int_fixpoint = int(round(0.716445 * pow(2, 53)))
print(int_fixpoint)

6453162870062910
